In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [207]:
df = pd.read_csv('2020-4-27.csv')
df.head()

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Views,Area,Road,Road Width,Road Type,Build Area,Posted,Amenities
0,Flat System House On Sale At Budhanikantha : H...,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000,6,3,2.0,10,West,2073.0,17,1-0-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,18-0-0-0 Aana,11 hours ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,21 Aana Beautiful House : House For Sale In Bu...,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000,5,3,2.0,9,East,2073.0,26,0-21-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-0-0 Aana,12 hours ago,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."
2,Beautiful To Floor House : House For Sale In D...,"Dhapasi, Dhapasi, Kathmandu",Kathmandu,70000000,5,3,2.0,12,East,2071.0,154,1-1-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,10-1-0-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
3,Modern Architecture Bungalow House : House For...,"baluwatar, Baluwatar, Kathmandu",Kathmandu,130000000,6,4,3.0,9,West,2017.0,17,0-10-1-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-7-1-0 Aana,1 day ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
4,Modern Bungalow House : House For Sale In Golf...,"Golfutar, Golfutar, Kathmandu",Kathmandu,100000000,6,3,2.0,10,East,2070.0,59,0-19-2-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-2-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back..."


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2209 entries, 0 to 2208
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       2209 non-null   object 
 1   Address     2209 non-null   object 
 2   City        2209 non-null   object 
 3   Price       2209 non-null   int64  
 4   Bedroom     2209 non-null   int64  
 5   Bathroom    2209 non-null   int64  
 6   Floors      1038 non-null   float64
 7   Parking     2209 non-null   int64  
 8   Face        2209 non-null   object 
 9   Year        582 non-null    float64
 10  Views       2209 non-null   object 
 11  Area        2209 non-null   object 
 12  Road        2209 non-null   object 
 13  Road Width  2209 non-null   object 
 14  Road Type   1424 non-null   object 
 15  Build Area  2209 non-null   object 
 16  Posted      2209 non-null   object 
 17  Amenities   2209 non-null   object 
dtypes: float64(2), int64(4), object(12)
memory usage: 310.8+ KB


In [209]:
null_counts = df.isnull().sum()
null_counts

Title            0
Address          0
City             0
Price            0
Bedroom          0
Bathroom         0
Floors        1171
Parking          0
Face             0
Year          1627
Views            0
Area             0
Road             0
Road Width       0
Road Type      785
Build Area       0
Posted           0
Amenities        0
dtype: int64

In [210]:
# calulate number of missing values for area and build area
def count_na_values(column):
    count = 0
    for value in column:
        if "N/A" in str(value):
            count += 1
    return count
na_counts = df[['Area','Build Area']].apply(count_na_values)
print(na_counts)

Area             0
Build Area    1770
dtype: int64


### Columns not used:
- title
- floors : 1172 out of 2209 values missing
- year : 1629 out of 2209 values missing
- road : dependent on road width and road type
- build area : 1772 out of 2209 values missing
- posted
- amenities: 1067 out of 2209 values missing

In [211]:
sel_cols = ['Address','City','Bedroom','Bathroom','Parking','Face','Views','Area','Road Width','Road Type']
X = df[sel_cols].copy()
y = df['Price']
print(X.tail())
print(y)

                                     Address       City  Bedroom  Bathroom  \
2204             Bhangal, Chapali, Kathmandu  Kathmandu        5         5   
2205          Nagarjun, Sitapaila, Kathmandu  Kathmandu        4         2   
2206  Deuja Chowk, Budhanilkantha, Kathmandu  Kathmandu        5         3   
2207       Bhimsengola, Baneshwor, Kathmandu  Kathmandu        9         3   
2208        Gocha marg, Baluwatar, Kathmandu  Kathmandu        5         2   

      Parking        Face  Views      Area Road Width Road Type  
2204        5  North East   3.1K  1 Ropani    16 Feet       NaN  
2205        1  North East   1.4K  2.5 Aana    13 Feet       NaN  
2206        1        East   1.5K    8 Aana    12 Feet       NaN  
2207        4       North   2.5K   51 Aana    13 Feet       NaN  
2208        5        East  19.6K   13 Aana    12 Feet       NaN  
0        90000000
1        80000000
2        70000000
3       130000000
4       100000000
          ...    
2204       500000
2205     

In [212]:
# keep the middle values of the address columns only
X['Address'] = X['Address'].str.split(',').str[1]
# print(X['Address'])

In [213]:
#convert views like 1k to 1000
def convert_to_numeric(value):
    if 'K' in value:
        return float(value.replace('K', '')) * 1000
    else:
        return float(value)

X['Views'] = X['Views'].apply(convert_to_numeric)
print(X['Views'])

0          17.0
1          26.0
2         154.0
3          17.0
4          59.0
         ...   
2204     3100.0
2205     1400.0
2206     1500.0
2207     2500.0
2208    19600.0
Name: Views, Length: 2209, dtype: float64


In [214]:
#convert all areas to squre feet

def remove_hyphen_or_dot(value):
    value = value.replace('-', '')
    value = value.replace('.', '')
    value = value.replace('/', '')
    value = value.replace(',', '')
    value = value.replace('+','')
    return value

def convert_to_square_feet(value):
    print(value)
    value = remove_hyphen_or_dot(value)
    if 'Aana' in value:
        return float(value.replace('Aana', '')) * 342.25      
    elif 'Ropani' in value:
        return float(value.replace('Ropani', '')) * 5476
    elif 'ropani aana' in value:
        return float(value.replace('ropani', '')) * 5476
    elif 'Kattha' in value:
        return float(value.replace('Kattha', '')) * 3645
    elif 'Dhur' in value:
        return float(value.replace('Dhur', '')) * 182.25
    elif 'Sq. Feet' in value:
        return float(value.replace('Sq. Feet', ''))
    else:
        return None

X['Area'] = X['Area'].apply(convert_to_square_feet)
print(X['Area'])

1-0-0-0 Aana
0-21-0-0 Aana
1-1-0-0 Aana
0-10-1-0 Aana
0-19-2-0 Aana
0-12-3-1 Aana
3-1-1 Aana
0-8-5-0 Aana
0-12-3-2 Aana
0-4-0-0 Aana
14 Aana
0-4-2-0 Aana
5 Aana
8 Aana
4 Aana
4 Aana
0-6-0-0 Aana
6 Aana
10 Aana
0-4-0-0 Aana
0-3-2-0 Aana
0-3-2-0 Aana
21 Aana
19 Aana
0-4-0-0 Aana
6.1 Aana
19 Aana
0-4-2-0 Aana
6.5 Aana
0-3-0-0 Aana
0-3-2-0 Aana
3-0-2 Aana
10 Aana
21 Aana
4 Ropani
4 Aana
7 Aana
5 Aana
2500 Sq. Feet
3.1 Aana
11 Aana
30 Aana
9 Aana
3.5 Aana
12 Aana
5 Aana
4 Aana
3.1 Aana
4 Aana
4 Aana
4.5 Aana
3.1 Aana
8 Aana
0-6-3-0 Aana
4 Aana
4.5 Aana
0-3-0-0 Aana
0-5-1-1 Aana
5 Aana
3.1.0 Aana
4-0-0-0 Ropani
0-9-3-3 Aana
0-4-2-0 Aana
7 Aana
0-3-2-0 Sq. Feet
0-0-11.5 Dhur
1-15-0-0 Aana
6 Aana
0-4-1-1 Aana
0-3-0-0 Aana
0-4-0-0 Aana
0-5-0-2 Aana
0-7-2-0 Aana
0-8-3-0 Sq. Feet
0-3-0-0 Aana
0-3-1-0 Aana
0-3-2-0 Aana
0-4-0-0 Aana
0-3-2-0 Aana
0-3-0-0 Aana
0-3-0-0 Aana
0-5-1-3 Aana
0-4-0-0 Aana
6.25 Dhur
0.4.0.0 Aana
0-4-0-2 Aana
0-6-2-0 Aana
0-4-0-0 Aana
0-3-2-2 Aana
3.3 Aana
0-6-1-0 Aana
0-10-0